<a href="https://colab.research.google.com/github/akshatjain-1/DETECTION_MITIGATION_NLP/blob/main/deBERTa_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install necessary libraries (if not already installed)
!pip install pandas numpy nltk sklearn



  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import pandas as pd  # For data manipulation
import numpy as np   # For numerical operations
import nltk          # For text preprocessing
from sklearn.model_selection import train_test_split  # For data splitting

# Download nltk datasets
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"akshatjain11111","key":"2e00c4118b5a8ec55df2602bd4abf04c"}'}

In [5]:
# Make a directory for Kaggle and move the kaggle.json file
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Change the permissions of the file so Kaggle can use it
!chmod 600 ~/.kaggle/kaggle.json


In [6]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge




 99% 52.0M/52.6M [00:03<00:00, 21.6MB/s]
100% 52.6M/52.6M [00:03<00:00, 14.4MB/s]


In [7]:
# Unzip the downloaded dataset
!unzip jigsaw-toxic-comment-classification-challenge.zip



Archive:  jigsaw-toxic-comment-classification-challenge.zip
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: test_labels.csv.zip     
  inflating: train.csv.zip           


In [8]:
!ls *.zip


jigsaw-toxic-comment-classification-challenge.zip  test.csv.zip		train.csv.zip
sample_submission.csv.zip			   test_labels.csv.zip


In [9]:
!unzip jigsaw-toxic-comment-classification-challenge.zip


Archive:  jigsaw-toxic-comment-classification-challenge.zip
replace sample_submission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: test_labels.csv.zip     
  inflating: train.csv.zip           


In [10]:
!unzip '*.zip'


Archive:  test.csv.zip
  inflating: test.csv                

Archive:  jigsaw-toxic-comment-classification-challenge.zip
replace sample_submission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: test_labels.csv.zip     
  inflating: train.csv.zip           

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  test_labels.csv.zip
  inflating: test_labels.csv         

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

5 archives were successfully processed.


In [11]:
!ls


jigsaw-toxic-comment-classification-challenge.zip  test.csv		train.csv
sample_data					   test.csv.zip		train.csv.zip
sample_submission.csv				   test_labels.csv
sample_submission.csv.zip			   test_labels.csv.zip


In [12]:
# Load the dataset (replace 'train.csv' with the actual CSV file name)
import pandas as pd
df = pd.read_csv('train.csv')

# Check the first few rows of the dataset
df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [13]:
df.head()

# Check for any missing values
print(df.isnull().sum())

# Get the shape of the dataset
print(f"The dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64
The dataset contains 159571 rows and 8 columns.


In [14]:
# Create a new column 'abusive' which is True if any of the labels is 1
df['abusive'] = df[['toxic', 'severe_toxic', 'identity_hate', 'insult', 'obscene', 'threat']].sum(axis=1) > 0

# Convert to binary (1 for abusive, 0 for non-abusive)
df['abusive'] = df['abusive'].astype(int)

# Inspect the new column
df[['abusive']].head()

,abusive
0,0
1,0
2,0
3,0
4,0


In [15]:
# Example preprocessing steps
import nltk
nltk.download('punkt')  # For tokenization
nltk.download('stopwords')  # For stopwords removal
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Convert text to lowercase
df['comment_text'] = df['comment_text'].str.lower()

# Tokenize the text
df['tokenized_text'] = df['comment_text'].apply(word_tokenize)

# Remove stopwords
stop_words = set(stopwords.words('english'))
df['cleaned_text'] = df['tokenized_text'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

# Remove special characters
import re
df['cleaned_text'] = df['cleaned_text'].apply(lambda tokens: [re.sub(r'[^a-zA-Z]', '', word) for word in tokens])

# Display the cleaned text
df[['cleaned_text']].head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,cleaned_text
0,"[explanation, edits, made, username, hardcore,..."
1,"[daww, , matches, background, colour, m, seemi..."
2,"[hey, man, , m, really, trying, edit, war, , s..."
3,"[, ca, nt, make, real, suggestions, improvemen..."
4,"[, sir, , hero, , chance, remember, page, s, ]"


In [16]:
# Join the tokens back into a string
df['cleaned_text'] = df['cleaned_text'].apply(lambda tokens: ' '.join(tokens))

# Display the cleaned text
df[['cleaned_text']].head()

,cleaned_text
0,explanation edits made username hardcore metal...
1,daww matches background colour m seemingly st...
2,hey man m really trying edit war s guy const...
3,ca nt make real suggestions improvement wond...
4,sir hero chance remember page s


In [17]:
from sklearn.model_selection import train_test_split

# Features (the cleaned text) and labels (abusive or multi-label)
X = df['cleaned_text']
y = df['abusive']  # or keep multiple labels for multi-label classification

df_sampled = df.sample(frac=0.2, random_state=42)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Check the sizes of the splits
print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")

Training samples: 111699
Testing samples: 47872


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=10000, stop_words='english')

# Fit the vectorizer on the training data and transform both the training and testing data
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Check the shape of the transformed data
print(f"TF-IDF matrix shape for training data: {X_train_tfidf.shape}")
print(f"TF-IDF matrix shape for testing data: {X_test_tfidf.shape}")

TF-IDF matrix shape for training data: (111699, 10000)
TF-IDF matrix shape for testing data: (47872, 10000)


In [20]:
# Download GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-09-18 12:58:55--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-09-18 12:58:56--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-09-18 12:58:56--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [21]:
import numpy as np

# Load the GloVe embeddings (100-dimensional version)
embedding_index = {}
with open("glove.6B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Create an embedding matrix for the words in your vocabulary
embedding_dim = 100  # 100d GloVe
vocab_size = len(tfidf.vocabulary_)
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Get the words from TF-IDF and build the embedding matrix
word_to_index = tfidf.vocabulary_  # Word-to-index mapping from TF-IDF
for word, i in word_to_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Check the embedding matrix shape
print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: (10000, 100)


In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

# Load deBERTa model and tokenizer
model_name = "microsoft/deberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # For binary classification

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['cleaned_text'], padding="max_length", truncation=True)

# Apply the tokenizer to the dataset
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128)

# Convert to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(y_train.values))
test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']), torch.tensor(y_test.values))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Tokenize the dataset
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128)

In [24]:
import torch

# Convert to PyTorch tensors
class ToxicDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ToxicDataset(train_encodings, y_train.values)
test_dataset = ToxicDataset(test_encodings, y_test.values)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load model

model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2)

# Define training arguments with efficient settings
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=4,   # Reduced batch size
    num_train_epochs=15,
    learning_rate=1e-5,

    weight_decay=0.01,
    fp16=True,                       # Enable mixed precision training
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,                # Log every 10 steps
)

# Define a callback to save the best model based on validation loss
from transformers import TrainerCallback

class SaveModelCheckpoint(TrainerCallback):
    def __init__(self):
        self.best_loss = float('inf')

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        current_loss = metrics['eval_loss']
        if current_loss < self.best_loss:
            self.best_loss = current_loss
            model.save_pretrained('./best_model')
            tokenizer.save_pretrained('./best_model')

# Initialize Trainer with callbacks
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[SaveModelCheckpoint()],
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

# Print memory usage
import psutil

def print_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"Memory Usage: {mem_info.rss / 1024**2:.2f} MB")

print_memory_usage()

